In [8]:
%pip install kfp

  Using cached kfp-2.12.1-py3-none-any.whl
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached kfp_pipeline_spec-0.6.0-py3-none-any.whl.metadata (293 bytes)
  Using cached kfp_server_api-2.4.0-py3-none-any.whl
  Using cached kubernetes-30.1.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.7.1-cp311-cp311-win_amd64.whl.metadata (2.4 kB)
Using cached kfp_pipeline_spec-0.6.0-py3-none-any.whl (9.1 kB)
Using cached docstring_parser-0.16-py3-none-any.whl (36 kB)
Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl (131 kB)
Using cached kubernet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.


In [9]:
from kfp import dsl
from kfp.dsl import *
from typing import *
from kfp import compiler

@dsl.component(packages_to_install=['pandas','scikit-learn','matplotlib','seaborn'])
def say_hello(name: str) -> str:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, classification_report
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, classification_report
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Create a synthetic dataset
    data = {
        'crop_name': ['wheat', 'rice', 'maize', 'wheat', 'rice', 'maize', 'wheat', 'rice', 'maize', 'wheat', 'rice', 'maize'],
        'temperature': [20, 25, 22, 21, 24, 23, 19, 26, 21, 20, 25, 22],
        'humidity': [30, 50, 45, 32, 48, 47, 31, 52, 44, 30, 50, 45],
        'soil_moisture': [40, 60, 55, 42, 58, 57, 41, 62, 54, 40, 60, 55],
        'disease_risk': ['low', 'high', 'medium', 'low', 'high', 'medium', 'low', 'high', 'medium', 'low', 'high', 'medium']
    }

    df = pd.DataFrame(data)

    # Display the first few rows of the dataset
    print(df.head())


    # Encode categorical variable for crop_name
    crop_label_encoder = LabelEncoder()
    df['crop_name'] = crop_label_encoder.fit_transform(df['crop_name'])

    # Encode target variable
    risk_label_encoder = LabelEncoder()
    df['disease_risk'] = risk_label_encoder.fit_transform(df['disease_risk'])

    # Features and target variable
    X = df[['crop_name', 'temperature', 'humidity', 'soil_moisture']]
    y = df['disease_risk']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    # Initialize the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train, y_train)


    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')

    # Detailed classification report
    print(classification_report(y_test, y_pred))

    # Plot feature importances
    feature_importances = model.feature_importances_
    features = X.columns

    # sns.barplot(x=feature_importances, y=features)
    # plt.xlabel('Importance')
    # plt.ylabel('Feature')
    # plt.title('Feature Importances')
    # plt.show()

    # Example prediction
    crop_name = 'rice'
    temperature = 25
    humidity = 50
    soil_moisture = 40

    crop_name_encoded = crop_label_encoder.transform([crop_name])[0]

    # Prepare the feature vector
    features = scaler.transform([[crop_name_encoded, temperature, humidity, soil_moisture]])

    # Predict the disease risk
    risk_encoded = model.predict(features)[0]

    # Decode the risk
    risk = risk_label_encoder.inverse_transform([risk_encoded])[0]
    print(f'The disease risk for {crop_name} is {risk}.')
    return risk

@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    return hello_task.output



compiler.Compiler().compile(hello_pipeline, 'disease_risk_pipeline02.yaml')

c:\Users\DELL\anaconda3\Lib\site-packages\kfp\dsl\component_decorator.py:126: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.9' to 'python:3.10' on Oct 1, 2025. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.10.
  return component_factory.create_component_from_func(
